In [1]:
import tensorflow as tf

In [2]:
TRAIN_TFRECORDS = './raw_data/train.tfrecords'
TEST_TFRECORDS = './raw_data/test.tfrecords'

In [3]:
def print_trainable_vals(sess):
    variables_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print('Variable: ', k, 'Shape: ', v.shape, 'Value: ', v)

In [4]:
class Model(object):
    
    def get_loss_acc(self, train_images, train_labels, reuse_variables=None):
        with tf.variable_scope(tf.get_variable_scope(), reuse=reuse_variables):
            y_ = self.inference(train_images)
            y = tf.to_float(train_labels)
            y.set_shape([128, 10])
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                    labels=y, logits=y_))
            correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return loss, accuracy


    def get_weight_variable(self, shape):
        weights = tf.get_variable("weights", shape,
                                  initializer=tf.truncated_normal_initializer(
                                          stddev=0.1))
        return weights


    def inference(self, train_images):
        with tf.variable_scope('layer1'):
            w = self.get_weight_variable([784, 10])
            b = tf.get_variable("biases", [10],
                                initializer=tf.constant_initializer(0.0))
            y = tf.matmul(train_images, w) + b
        return y


    def train(self, file_names, global_step):
        with tf.variable_scope(tf.get_variable_scope()):
            samples = input_fn(file_names, 128)
            train_images, train_labels = samples
            loss, acc = self.get_loss_acc(train_images, train_labels)
            opt = tf.train.GradientDescentOptimizer(0.5)
            train_op = opt.minimize(loss, global_step=global_step)
            return train_op, loss, acc



In [5]:
def generate_parse_fn(batch_size):
    def read_examples(examples):
        features = {}
        features['label'] = tf.FixedLenFeature([], tf.int64)
        features['image_raw'] = tf.FixedLenFeature([], tf.string)
        features = tf.parse_example(examples, features)
        images = tf.decode_raw(features['image_raw'], tf.uint8)
        images.set_shape([batch_size, 784])
        images = tf.cast(images, tf.float32) * (1. / 255) - 0.5
        labels = features['label']
        one_hot_labels = tf.to_float(tf.one_hot(labels, 10, 1, 0))
        return images, one_hot_labels

    return read_examples


def input_fn(file_names, batch_size, epoch=None):
    _parse_fn = generate_parse_fn(batch_size)
    files = tf.data.Dataset.list_files(file_names)
    # number_of_cpu is the value of worker.vcore in xml file
    dataset = files.apply(tf.contrib.data.parallel_interleave(
            tf.data.TFRecordDataset,
            cycle_length=4 * 2))
    # prefetch will buffer the previos op and improve the performance
    dataset = dataset.prefetch(buffer_size=batch_size)
    # times: user defined
    dataset = dataset.shuffle(buffer_size=batch_size * 4)
    # buffer the shuffle op and improve the perfromance
    dataset = dataset.prefetch(buffer_size=batch_size)
    dataset = dataset.repeat(epoch)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(_parse_fn, num_parallel_calls=4)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [6]:
global_step = tf.train.get_or_create_global_step()
model_a = Model()
train_op, loss, acc = model_a.train(TRAIN_TFRECORDS, global_step)
tf.get_variable_scope().reuse_variables()
model_b = Model()
_, valid_loss, valid_acc = model_a.train(TEST_TFRECORDS,global_step)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [7]:
print(tf.global_variables())

[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>, <tf.Variable 'layer1/weights:0' shape=(784, 10) dtype=float32_ref>, <tf.Variable 'layer1/biases:0' shape=(10,) dtype=float32_ref>]


In [12]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for __ in range(100):
    _, loss_val, acc_val, step = sess.run([train_op, loss, acc, global_step])
#     print('loss: ', loss_val, ' acc_val: ', acc_val, ' step: ', step)
print_trainable_vals(sess)
print('........................................training done........................................')
for __ in range(100):
    loss_val, acc_val, step = sess.run([valid_loss, valid_acc, global_step])
#     print('loss: ', loss_val, ' acc_val: ', acc_val, ' step: ', step) 

print_trainable_vals(sess)

Variable:  layer1/weights:0 Shape:  (784, 10) Value:  [[-0.02052911  0.11269705  0.08209549 ...,  0.09361597  0.08152776
  -0.04943234]
 [ 0.13435555  0.0461245   0.04200288 ...,  0.03626484  0.01160972
  -0.0092969 ]
 [ 0.04860592 -0.03488415  0.07515283 ..., -0.02267688 -0.01706903
  -0.00920649]
 ..., 
 [ 0.07202993  0.01894343 -0.06945126 ...,  0.02244743 -0.07327471
  -0.07766617]
 [-0.10550149 -0.01919532  0.07699977 ..., -0.15903479 -0.12641491
   0.04359466]
 [-0.00423116  0.09942098 -0.0248875  ..., -0.03972405 -0.03542226
   0.06878631]]
Variable:  layer1/biases:0 Shape:  (10,) Value:  [ 0.02276097 -0.06405236  0.03249261  0.02673912 -0.0010454   0.01680639
  0.00035756 -0.01868777  0.03820047 -0.05357032]
........................................training done........................................
Variable:  layer1/weights:0 Shape:  (784, 10) Value:  [[-0.02052911  0.11269705  0.08209549 ...,  0.09361597  0.08152776
  -0.04943234]
 [ 0.13435555  0.0461245   0.04200288 ...,  

In [9]:
print(tf.global_variables())

[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>, <tf.Variable 'layer1/weights:0' shape=(784, 10) dtype=float32_ref>, <tf.Variable 'layer1/biases:0' shape=(10,) dtype=float32_ref>]
